# Final: Classification with YOLOv8

In [4]:
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from ultralytics import YOLO
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# setting random seed
random_seed = 42
name = f"seed{random_seed}"
custom_epochs = 40

# source folder and classes
source_dir = Path("/home/shared-data/corrosion_images")
classes = ["corrosion", "no_corrosion"]

# new folder structure to be created
output_dir = Path("/home/liva/classification_dataset")
train_dir = output_dir / "train"
val_dir = output_dir / "val"
test_dir = output_dir / "test"

## 1. Create Train/Val Dataset

In [3]:
# create directories
for split in [train_dir, val_dir, test_dir]:
    for cls in classes:
        (split / cls).mkdir(parents=True, exist_ok=True)

# split and copy images
for cls in classes:
    cls_path = source_dir / cls
    images = (
        list(cls_path.glob("*.jpg")) +
        list(cls_path.glob("*.png")) +
        list(cls_path.glob("*.jpeg"))
    )

    # 70% train, 30% temp
    train_imgs, temp_imgs = train_test_split(
        images,
        test_size=0.3,
        random_state=random_seed,
        shuffle=True
    )

    # split temp into val and test (15% each)
    val_imgs, test_imgs = train_test_split(
        temp_imgs,
        test_size=0.5,
        random_state=random_seed,
        shuffle=True
    )

    # copy train images
    for img in train_imgs:
        shutil.copy2(img, train_dir / cls / img.name)

    # copy validation images
    for img in val_imgs:
        shutil.copy2(img, val_dir / cls / img.name)

    # copy test images
    for img in test_imgs:
        shutil.copy2(img, test_dir / cls / img.name)

# counts
print(f"Train set: {len(list(train_dir.rglob('*.*')))} images")
print(f"Val set: {len(list(val_dir.rglob('*.*')))} images")
print(f"Test set: {len(list(test_dir.rglob('*.*')))} images")

Train set: 2799 images
Val set: 599 images
Test set: 601 images


## Create Funciton to Save Training and Validation Loss Plot

I will now create a function, that will plot our training and validation loss when called.

In [14]:
def plot_train_val_loss(csv_path, title=None, save_path=None):
    df = pd.read_csv(csv_path)

    if title is None:
        title = "Training vs Validation Loss"

    # create figure
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(df['epoch'], df['train/loss'], label='Training Loss')
    ax.plot(df['epoch'], df['val/loss'], label='Validation Loss')

    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title(title)
    ax.legend()
    ax.grid(True)

    # save plot
    if save_path:
        if not save_path.lower().endswith(".png"):
            save_path += ".png"
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")

    plt.close(fig)

## Baseline Highway data

In [5]:
# classes
classes = ["no_corrosion", "corrosion"]
class_to_idx = {cls: i for i, cls in enumerate(classes)}

# initialize lists
images = []
labels = []

for cls in classes:
    cls_folder = Path(test_dir) / cls
    for img_path in cls_folder.glob("*"):
        if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            images.append(str(img_path))
            labels.append(class_to_idx[cls])

labels = np.array(labels)

# generating random predictions
np.random.seed(42)
random_preds = np.random.randint(0, len(classes), size=len(images))

# calculating accuracy and creating confusion matrix
accuracy = accuracy_score(labels, random_preds)
cm = confusion_matrix(labels, random_preds)

print(f"Accuracy: {accuracy*100:.2f}%")

print("Confusion Matrix:\n", cm)


Accuracy: 52.58%
Confusion Matrix:
 [[153 149]
 [136 163]]


Baseline Accuracy is 52.58%

## Training Final YOLOv8s Model 

For that, I need new epochs, as I will train with 40 epochs.

In [ ]:
model_small_final = YOLO("yolov8s-cls.pt") # small
results_small = model_small_final.train(
    data=output_dir,
    epochs=custom_epochs,
    imgsz=256,
    batch=64,
    device='0',
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.1,
    momentum=0.9,
    weight_decay=0.0005,
    dropout=0.2,
    patience=10,
    name=f"0000{name}_{custom_epochs}epochs_yoloS_final",
    seed=random_seed
)

      29/40      2.34G    0.02661         46        256: 100% ━━━━━━━━━━━━ 44/44 2.3it/s 19.3s0.3s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 5/5 20.3it/s 0.2s.1s
                   all      0.983          1

      Epoch    GPU_mem       loss  Instances       Size
      30/40      2.34G    0.01652         64        256: 73% ━━━━━━━━╸─── 32/44 10.4it/s 13.9s<1.2s

Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


      30/40      2.34G     0.0173         46        256: 100% ━━━━━━━━━━━━ 44/44 2.2it/s 20.1s0.3ss
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 5/5 20.2it/s 0.2s.1s
                   all      0.987          1

      Epoch    GPU_mem       loss  Instances       Size
      31/40      2.34G    0.01229         64        256: 52% ━━━━━━────── 23/44 10.7it/s 10.6s<2.0s

Invalid SOS parameters for sequential JPEG


      31/40      2.34G    0.01212         46        256: 100% ━━━━━━━━━━━━ 44/44 2.1it/s 21.1s0.3ss
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 5/5 20.0it/s 0.3s.1s
                   all      0.985          1
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 21, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

31 epochs completed in 0.176 hours.
Optimizer stripped from /home/liva/runs/classify/0000seed42_40epochs_yoloS_final/weights/last.pt, 10.3MB
Optimizer stripped from /home/liva/runs/classify/0000seed42_40epochs_yoloS_final/weights/best.pt, 10.3MB

Validating /home/liva/runs/classify/0000seed42_40epochs_yoloS_final/weights/best.pt...
Ultralytics 8.3.217 🚀 Python-3.10.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14916MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,077,762 parameters, 0 gradients,

Invalid SOS parameters for sequential JPEG


The model stopped earlier (epoch 31) due to patience.

In [15]:
plot_train_val_loss("runs/classify/0000seed42_40epochs_yoloS_final/results.csv", title="Training vs Validation Loss with YOLOv8", save_path="output/yolo_finalmodel")

Plot saved to: output/yolo_finalmodel.png


Now, we want to look at the accuracy: 

In [ ]:
model = YOLO("/home/liva/runs/classify/0000seed42_40epochs_yoloS_final/weights/best.pt")

In [9]:
model.val(data=test_dir)

Ultralytics 8.3.217 🚀 Python-3.10.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14916MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,077,762 parameters, 0 gradients, 12.4 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /home/liva/classification_dataset/test/train
Found 601 images in subdirectories. Attempting to split...
Splitting /home/liva/classification_dataset/test (2 classes, 601 images) into 80% train, 20% val...
Split complete in /home/liva/classification_dataset/test_split ✅
train: /home/liva/classification_dataset/test_split/train... found 601 images in 2 classes ✅ 
val: /home/liva/classification_dataset/test_split/val... found 502 images in 2 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3390.8±1149.5 MB/s, size: 711.1 KB)
val: Scanning /home/liva/classification_dataset/test_split/val... 502 images, 0 corrupt: 100% ━━━━━━━━━━━━ 502/502 2.1Kit/s 0.2s0.1ss
val: New cache created: /home/liva/classification_dataset/test_split/val.cache
               cla

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f060ceae5f0>
curves: []
curves_results: []
fitness: 0.9910358488559723
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9820716977119446, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9910358488559723}
save_dir: PosixPath('/home/liva/runs/classify/val18')
speed: {'preprocess': 0.12125013269988663, 'inference': 1.0163670042121553, 'loss': 0.0010101770262319254, 'postprocess': 0.001265982767025313}
task: 'classify'
top1: 0.9820716977119446
top5: 1.0

The accuracy of this model is 0.9835. (might be slightly different in above output (top1))
Comparing that to the Baseline Accuracy of 52.58%, we have a much better classification model.

But this data is still only the highway training data. Our use-case is corrosion damages in ports and waterways, therefore I will test how well this training set is holding up against a test set of use-case data:

## Applying our model on Use-Case Data

In [ ]:
model = YOLO("/home/liva/runs/classify/0000seed42_40epochs_yoloS_final/weights/best.pt")
# evaluate on the use case images
metrics = model.val(data="/home/liva/use_case_usecase.yaml")

Ultralytics 8.3.217 🚀 Python-3.10.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14916MiB)


YOLOv8s-cls summary (fused): 30 layers, 5,077,762 parameters, 0 gradients, 12.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4371.1±1274.6 MB/s, size: 6029.2 KB)
val: Scanning /home/liva/use_case_imgs... 73 images, 0 corrupt: 100% ━━━━━━━━━━━━ 73/73 1.6Kit/s 0.0s
val: New cache created: /home/liva/use_case_imgs.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 5/5 1.2it/s 4.3s0.2ss
                   all      0.795          1
Speed: 1.3ms preprocess, 3.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/liva/runs/classify/val20


The use-case accuracy is 0.795.

## Baseline Harbor Use-Case Data

In [22]:
use_case_dir = "/home/liva/use_case_imgs"

# initialize lists
images = []
labels = []

for cls in classes:
    cls_folder = Path(use_case_dir) / cls
    for img_path in cls_folder.glob("*"):
        if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            images.append(str(img_path))
            labels.append(class_to_idx[cls])

labels = np.array(labels)

# generating random predictions
np.random.seed(42)
random_preds = np.random.randint(0, len(classes), size=len(images))

# calculating accuracy and creating confusion matrix
accuracy = accuracy_score(labels, random_preds)
cm = confusion_matrix(labels, random_preds)

print(f"Accuracy: {accuracy*100:.2f}%")

print("Confusion Matrix:\n", cm)


Accuracy: 60.27%
Confusion Matrix:
 [[11  4]
 [25 33]]


Comparing the 79.5% accuracy of the model to the baseline results of 60.27% shows that the model classifies significantly better.